# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import pandas as pd
import numpy as np
import json
import os
import scipy.stats as stats
import seaborn as sns

In [ ]:
with open('../params.json', 'r') as file :
    params = json.load(file)

DATASET, VERSION = params['dataset'], params['version']
DATA_FOLD = params['data_folder']

In [ ]:
DIR = f'{DATA_FOLD}/{VERSION}/3.analysis/imputation_48/{DATASET}/tables/'
print({DATASET})

# Datasets

In [ ]:
df_mean = pd.read_excel(DIR + '/pam_comparaison/' + 'pa_only_mean.xlsx')
df_lin = pd.read_excel(DIR + '/pam_comparaison/' + 'pa_only_lin_interpol.xlsx')
df_saits = pd.read_excel(DIR + '/pam_comparaison/' + 'pa_only_saits.xlsx')

In [ ]:
df_mean.shape == df_lin.shape == df_saits.shape

#  Bland & Altman function

In [ ]:
df_mean

In [ ]:
sns.set_style('whitegrid')

fig, axes = plt.subplots(3, figsize = (16,12))

datasets = [
    (df_mean, "A : Mean", axes[0]),
    (df_lin, "B : Linear Interpolation", axes[1]),
    (df_saits, "C : SAITS", axes[2]),
]

for df, title, ax in datasets:
    x = 'moyenne'
    y= 'différence'
    values = np.vstack([df[x], df[y]])
    kernel = stats.gaussian_kde(values)(values)
    sns.scatterplot(data = df, x=x, y=y, ax=ax, c=kernel, cmap='viridis')
    #twin_ax= ax.twinx()
    #sns.kdeplot(data=df, x=x, ax=twin_ax, color='green')
    #sns.scatterplot(data = df, x='masquées', y='différence', ax=ax, alpha=0.5)
    ax.set_ylim(-100, 80)
    ax.set_xlim(20, 140)
    #twin_ax.set_ylim(0,0.2)
    ax.set_xlabel('Average of Imputed and Msked Values (mmHg)')
    ax.set_ylabel('Imputed Value - Masked Values (mmHg)')
    ax.set_title(title, loc='left')

In [ ]:
def plot_bland_altman_joint(df, title, bins=10):
    """
    Affiche une figure de Bland-Altman avec jointplot utilisant :
      - l'axe x = différence (imputation - valeur masquée)
      - l'axe y = moyenne (imputation et masquée)
      - des statistiques locales (moyenne et ±1.96 SD) calculées dans des bins de x

    Paramètres:
      df    : DataFrame contenant les colonnes 'différence' et 'moyenne'
      title : Titre de la figure
      bins  : Nombre de bins à utiliser pour le calcul des statistiques locales (défaut 10)
    """
    # Affectation des variables
    x = df['moyenne']
    y = df['différence']
    
    # Création du jointplot : scatter central et distributions marginales
    g = sns.jointplot(x=x, y=y, kind='scatter', height=6,)
                     # marginal_kws=dict(bins=20, fill=True))
    g.figure.suptitle(title, fontsize=16)
    # Ajustement pour éviter que le titre ne chevauche le plot
    g.figure.subplots_adjust(top=0.92)
    
    # Calcul des statistiques locales par bins
    # On définit des intervalles sur la plage de x
    bins_edges = np.linspace(x.min(), x.max(), bins+1)
    # On affecte chaque point à un bin via np.digitize (les bins vont de 1 à bins)
    bin_idx = np.digitize(x, bins_edges)
    
    # Tableaux pour stocker les centres de bins, les moyennes locales et les SD locales
    bin_centers = []
    local_means = []
    local_upper = []
    local_lower = []
    
    # Pour chaque bin, on calcule la moyenne (y) et l'écart-type
    for i in range(1, bins+1):
        mask = bin_idx == i
        if np.sum(mask) > 0:
            # Option 1 : centre calculé sur les x du bin
            bin_center = x[mask].mean()
            # Option 2 (alternative) : milieu du bin : (bins_edges[i-1] + bins_edges[i]) / 2
            mean_y = y[mask].mean()
            std_y = y[mask].std(ddof=1)
            bin_centers.append(bin_center)
            local_means.append(mean_y)
            local_upper.append(mean_y + 1.96 * std_y)
            local_lower.append(mean_y - 1.96 * std_y)
    
    # Récupération de l'axe principal (centre du jointplot)
    ax = g.ax_joint
    # Tracé de la courbe de la moyenne locale et des limites ±1.96 SD
    ax.plot(bin_centers, local_means, color='red', linestyle='-', label='Moyenne locale')
    ax.plot(bin_centers, local_upper, color='grey', linestyle='--', label='Moyenne +1.96 SD')
    ax.plot(bin_centers, local_lower, color='grey', linestyle='--', label='Moyenne -1.96 SD')
    
    ax.legend(loc='best')
    
    # Mise à jour des labels avec la bonne assignation des variables
    ax.set_xlabel("Moyenne (Imputation & Masquée) (mmHg)")
    ax.set_ylabel("Différence (Imputation - Masquée) (mmHg)")

    
    plt.show()

# Exemple d'utilisation avec trois jeux de données
# Remplace df_mimic_mean, df_mimic_lin, df_mimic_saits par tes DataFrames
plot_bland_altman_joint(df_mean, "A : Mean Imputation")
plot_bland_altman_joint(df_lin, "B : Linear Interpolation")
plot_bland_altman_joint(df_saits, "C : SAITS")